# Проект MobileGame
## Задание 1

### 1 Когортный анализ и расчет retention

In [1]:
# Импортируем библиотеки
import pandas as pd
import datetime as dt
import requests
from urllib.parse import urlencode

In [2]:
# Создаем функцию для загрузки данных с яндекс-диска
def yandex_read_csv(public_key):
    # используем api
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    # получаем url
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    # загружаем файл в df
    download_response = requests.get(download_url)
    df = pd.read_csv(download_url, sep=';')
    return df

In [3]:
# Загружаем данные
reg_data = yandex_read_csv('https://disk.yandex.ru/d/SRs6qGjPFc2GJg') # данные о времени регистрации
auth_data = yandex_read_csv('https://disk.yandex.ru/d/ytkERKye_ZFK5A') # данные о времени захода пользователей в игру

In [4]:
# Посмотрим как выглядят данные о времени регистрации
reg_data.head(2)

,reg_ts,uid
0,911382223,1
1,932683089,2


In [5]:
# Размер таблицы
print('Размер таблицы:', reg_data.shape)
# Типы данных
print ('Типы данных:\n', reg_data.dtypes)

Размер таблицы: (1000000, 2)
Типы данных:
 reg_ts    int64
uid       int64
dtype: object


In [6]:
# Посмотрим как выглядят данные о времени захода пользователей в игру
auth_data.head(2)

,auth_ts,uid
0,911382223,1
1,932683089,2


In [7]:
# Размер таблицы
print('Размер таблицы:', auth_data.shape)
# Типы данных
print ('Типы данных:\n', auth_data.dtypes)

Размер таблицы: (9601013, 2)
Типы данных:
 auth_ts    int64
uid        int64
dtype: object


Описание данных:
- reg_ts - дата регистрации
- uid - идентификатор игрока
- auth_ts - дата захода пользователей в игру

Так как период для анализа слишком велик (более 20 лет), необходимо ограничить диапазон анализа. Ограничить необходимо как дату регистрации, так и дату захода пользователей в игру. Связано это с тем, что могут быть пользователи, которые играют длительный промежуток времени

In [8]:
# Посмотрим за какой период у нас сейчас есть данные
print('Дата первой регистрации:', dt.datetime.utcfromtimestamp(reg_data.reg_ts.min()).strftime('%Y-%m-%d'))
print('Дата последней регистрации:', dt.datetime.utcfromtimestamp(reg_data.reg_ts.max()).strftime('%Y-%m-%d'))
print('Дата первого захода пользователя в игру:', dt.datetime.utcfromtimestamp(auth_data.auth_ts.min()).strftime('%Y-%m-%d'))
print('Дата последнего захода пользователя в игру:', dt.datetime.utcfromtimestamp(auth_data.auth_ts.max()).strftime('%Y-%m-%d'))

Дата первой регистрации: 1998-11-18
Дата последней регистрации: 2020-09-23
Дата первого захода пользователя в игру: 1998-11-18
Дата последнего захода пользователя в игру: 2020-09-23


In [9]:
# В качестве тестового расчета возьмем даты последнего месяца. 
date_start = '2020-09-01'
date_end = '2020-09-24'  # Прибавляем 1 день к последнему дню, так как при переводе в timestamp отсутствуют часы и минуты.
                         # Если указать 2020-09-23 то не будут учтены данные после 00:00.

In [10]:
# Переведем даты в формат timestamp, так как данные формата int будут обрабатываться быстрее, чем формата date 
date_start = dt.datetime.strptime(date_start, "%Y-%m-%d").timestamp()
date_end = dt.datetime.strptime(date_end, "%Y-%m-%d").timestamp()

Пользователь может начать играть только после регистрации. Оставим только данные, в которых время регистрации и время захода больше времени, с которого мы начинаем анализировать. Таким образом, мы уменьшим таблицы и ускорим процесс вычисления

In [11]:
reg_data = reg_data.query('reg_ts >= @date_start')
auth_data = auth_data.query('auth_ts >= @date_start')

In [12]:
# Объединяем данные по общему полю. 
cohort = reg_data.merge(auth_data, how='left', on='uid')

In [13]:
# Ограничим анализируемы период захода пользователей в игру
cohort = cohort.query('auth_ts <= @date_end')

In [14]:
# Изменим тип даты из timestamp в date (retention для примера расчитаем в днях)
cohort.reg_ts = cohort.reg_ts.apply(lambda x: dt.datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
cohort.auth_ts = cohort.auth_ts.apply(lambda x: dt.datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))

In [15]:
# Посчитаем количество уникальных пользователей
cohort = cohort.groupby(['reg_ts', 'auth_ts'], as_index=False).agg({'uid': 'nunique'}) 

In [16]:
# Добавим в таблицу столбец с периодом (номерация от первого заказа до последующего)
cohort['period'] = cohort.sort_values(['reg_ts', 'auth_ts'], ascending=True).groupby(['reg_ts']).cumcount()

In [17]:
# Добавим в таблицу столбец retention
cohort['retention'] = cohort.uid / cohort.groupby('reg_ts')['uid'].transform('max') 

In [18]:
# Приведем в читаемый вид
retention = cohort.pivot(index='reg_ts', columns='period', values='retention')

In [19]:
ur_style = (retention
            .style
            .set_caption('User retention by cohort')  # добавляем подпись
            .background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту
ur_style

period,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
reg_ts,,,,,,,,,,,,,,,,,,,,,,,
2020-08-31,100.00%,5.05%,3.03%,5.56%,6.57%,7.58%,8.59%,2.53%,5.56%,6.57%,7.07%,4.04%,4.04%,5.05%,6.57%,4.55%,5.56%,4.04%,3.54%,4.04%,3.54%,3.54%,2.02%
2020-09-01,100.00%,2.02%,4.10%,4.22%,4.60%,6.43%,5.61%,6.43%,4.10%,5.42%,5.11%,5.36%,4.85%,5.49%,4.29%,4.41%,4.35%,5.04%,4.16%,4.35%,3.59%,4.35%,1.58%
2020-09-02,100.00%,2.52%,3.90%,4.66%,5.67%,6.23%,7.12%,6.30%,4.85%,4.09%,5.67%,5.48%,5.48%,4.35%,4.97%,5.29%,4.16%,4.85%,3.90%,3.72%,4.66%,2.64%,
2020-09-03,100.00%,2.33%,5.41%,4.71%,6.54%,6.98%,8.55%,6.35%,5.47%,6.29%,6.35%,5.53%,5.85%,5.97%,5.47%,5.34%,4.40%,4.84%,4.65%,5.47%,2.39%,,
2020-09-04,100.00%,2.01%,3.64%,3.64%,5.15%,6.40%,7.28%,4.96%,5.34%,4.33%,5.65%,5.52%,4.33%,5.59%,4.39%,4.52%,4.52%,4.27%,4.77%,2.64%,,,
2020-09-05,100.00%,2.76%,3.95%,4.64%,5.45%,5.89%,7.46%,4.89%,5.08%,5.45%,6.02%,5.20%,4.57%,5.39%,4.51%,5.14%,4.45%,4.76%,2.51%,,,,
2020-09-06,100.00%,3.13%,4.32%,5.44%,5.63%,5.88%,6.82%,6.07%,5.82%,5.50%,4.88%,4.75%,5.38%,5.19%,5.50%,4.57%,4.82%,2.56%,,,,,
2020-09-07,100.00%,2.94%,4.25%,5.37%,5.06%,6.37%,7.31%,5.25%,4.12%,5.62%,6.00%,5.68%,4.56%,5.06%,4.25%,5.06%,3.37%,,,,,,
2020-09-08,100.00%,1.75%,3.93%,3.93%,4.86%,7.61%,7.54%,6.61%,5.17%,5.17%,6.55%,5.36%,5.92%,5.17%,4.74%,3.05%,,,,,,,


### 2 Напишем функцию для подсчета retention

In [1]:
def retantion(date_start, date_end, details):
    # функция для подсчета retantion 
    '''
    На вход в функцию подаются данные:

    date_start - дата начала анализируемого периода
    date_end - дата окончания анализируемого периода + 1 день
    details - детализация анализа:
        - 'day' - день
        - 'mounth' - месяц
        - 'year' - год
    '''
    
    # проверка правильности задания переменной details
    if details == 'day':
        details_2 = "%Y-%m-%d"
    elif details == 'mounth':
        details_2 = "%Y-%m"
    elif details == 'year':
        details_2 = "%Y"
    else:
        print('Переменная details задана неверно')
        return
    
    # импортируем нужные библиотеки
    import pandas as pd
    import datetime as dt
    
    # проверка правильности указания дат
    if dt.datetime.strptime(date_start, "%Y-%m-%d").timestamp() > dt.datetime.strptime(date_end, "%Y-%m-%d").timestamp():
        print('Даты перепутаны местами')
        return
    
    # считывание файлов
    reg_data = pd.read_csv('shared/problem1-reg_data.csv', sep=';') 
    auth_data = pd.read_csv('shared/problem1-auth_data.csv', sep=';') 
    
    # проверка правильности указания дат
    if date_start < dt.datetime.utcfromtimestamp(reg_data.reg_ts.min()).strftime('%Y-%m-%d'):
        print('Переменная date_start задана неверно. Дата первой регистрации: 1998-11-18')
        return
    if date_start > (dt.datetime.utcfromtimestamp(reg_data.reg_ts.max()) + dt.timedelta(days=1)).strftime('%Y-%m-%d'):
        print('Переменная date_start задана неверно. Дата последней регистрации: 2020-09-23')
        return
    if date_end < dt.datetime.utcfromtimestamp(auth_data.auth_ts.max()).strftime('%Y-%m-%d'):
        print('Переменная date_end задана неверно. Дата первого захода пользователя в игру: 1998-11-18')
        return
    if date_end > (dt.datetime.utcfromtimestamp(auth_data.auth_ts.max()) + dt.timedelta(days=1)).strftime('%Y-%m-%d'):
        print('Переменная date_end задана неверно. Дата последнего захода пользователя в игру: 2020-09-23')
        return
    
    # переведем даты в формат timestamp
    date_start = dt.datetime.strptime(date_start, "%Y-%m-%d").timestamp()
    date_end = dt.datetime.strptime(date_end, "%Y-%m-%d").timestamp()
    
    # ограничиваем анализируемый период
    reg_data = reg_data.query('reg_ts >= @date_start')
    auth_data = auth_data.query('auth_ts >= @date_start')
    
    # объединяем таблицы
    cohort = reg_data.merge(auth_data, how='left', on='uid')
    
    # ограничиваем анализируемый период
    cohort = cohort.query('auth_ts <= @date_end')
    
    # меняем тип даты из timestamp в date
    cohort.reg_ts = cohort.reg_ts.apply(lambda x: dt.datetime.utcfromtimestamp(x).strftime(details_2))
    cohort.auth_ts = cohort.auth_ts.apply(lambda x: dt.datetime.utcfromtimestamp(x).strftime(details_2))
    
    # считаем количество уникальных пользователей
    cohort = cohort.groupby(['reg_ts', 'auth_ts'], as_index=False).agg({'uid': 'nunique'}) 
    
    # добавляем столбец с периодом
    cohort['period'] = cohort.sort_values(['reg_ts', 'auth_ts'], ascending=True).groupby(['reg_ts']).cumcount()
    
    # добавляем в таблицу столбец retention
    cohort['retention'] = cohort.uid / cohort.groupby('reg_ts')['uid'].transform('max') 
    
    # приводим в читаемый вид
    retention = cohort.pivot(index='reg_ts', columns='period', values='retention')
    
    # визуализируем
    ur_style = (retention
            .style
            .set_caption(f"User retention by cohort in {details}")  # добавляем подпись
            .background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту
    return ur_style

In [2]:
# Проверим расчет retention в днях 
retantion('2020-09-01', '2020-09-24', 'day')

period,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
reg_ts,,,,,,,,,,,,,,,,,,,,,,,
2020-08-31,100.00%,5.05%,3.03%,5.56%,6.57%,7.58%,8.59%,2.53%,5.56%,6.57%,7.07%,4.04%,4.04%,5.05%,6.57%,4.55%,5.56%,4.04%,3.54%,4.04%,3.54%,3.54%,2.02%
2020-09-01,100.00%,2.02%,4.10%,4.22%,4.60%,6.43%,5.61%,6.43%,4.10%,5.42%,5.11%,5.36%,4.85%,5.49%,4.29%,4.41%,4.35%,5.04%,4.16%,4.35%,3.59%,4.35%,1.58%
2020-09-02,100.00%,2.52%,3.90%,4.66%,5.67%,6.23%,7.12%,6.30%,4.85%,4.09%,5.67%,5.48%,5.48%,4.35%,4.97%,5.29%,4.16%,4.85%,3.90%,3.72%,4.66%,2.64%,
2020-09-03,100.00%,2.33%,5.41%,4.71%,6.54%,6.98%,8.55%,6.35%,5.47%,6.29%,6.35%,5.53%,5.85%,5.97%,5.47%,5.34%,4.40%,4.84%,4.65%,5.47%,2.39%,,
2020-09-04,100.00%,2.01%,3.64%,3.64%,5.15%,6.40%,7.28%,4.96%,5.34%,4.33%,5.65%,5.52%,4.33%,5.59%,4.39%,4.52%,4.52%,4.27%,4.77%,2.64%,,,
2020-09-05,100.00%,2.76%,3.95%,4.64%,5.45%,5.89%,7.46%,4.89%,5.08%,5.45%,6.02%,5.20%,4.57%,5.39%,4.51%,5.14%,4.45%,4.76%,2.51%,,,,
2020-09-06,100.00%,3.13%,4.32%,5.44%,5.63%,5.88%,6.82%,6.07%,5.82%,5.50%,4.88%,4.75%,5.38%,5.19%,5.50%,4.57%,4.82%,2.56%,,,,,
2020-09-07,100.00%,2.94%,4.25%,5.37%,5.06%,6.37%,7.31%,5.25%,4.12%,5.62%,6.00%,5.68%,4.56%,5.06%,4.25%,5.06%,3.37%,,,,,,
2020-09-08,100.00%,1.75%,3.93%,3.93%,4.86%,7.61%,7.54%,6.61%,5.17%,5.17%,6.55%,5.36%,5.92%,5.17%,4.74%,3.05%,,,,,,,


In [3]:
# Проверим расчет retention в месяцах
retantion('2020-01-01', '2020-09-24', 'mounth')

period,0,1,2,3,4,5,6,7,8,9
reg_ts,,,,,,,,,,
2019-12,100.00%,25.56%,9.02%,6.77%,6.77%,6.77%,6.77%,6.77%,6.77%,6.77%
2020-01,100.00%,17.01%,6.42%,5.01%,5.01%,5.01%,5.01%,5.01%,5.01%,
2020-02,100.00%,17.69%,6.06%,4.92%,4.92%,4.92%,4.92%,4.92%,,
2020-03,100.00%,17.40%,6.28%,5.12%,5.12%,5.12%,5.12%,,,
2020-04,100.00%,17.92%,6.21%,5.11%,5.11%,5.11%,,,,
2020-05,100.00%,17.55%,6.46%,5.13%,5.13%,,,,,
2020-06,100.00%,17.50%,6.08%,4.91%,,,,,,
2020-07,100.00%,17.49%,6.05%,,,,,,,
2020-08,100.00%,17.52%,,,,,,,,


In [4]:
# Проверим расчет retention в годах
retantion('2015-01-01', '2020-09-24', 'year')

period,0,1,2,3,4,5
reg_ts,,,,,,
2014,100.00%,,,,,
2015,100.00%,6.34%,4.90%,4.90%,4.90%,4.90%
2016,100.00%,6.49%,5.01%,5.01%,5.01%,
2017,100.00%,6.55%,5.03%,5.03%,,
2018,100.00%,6.50%,5.01%,,,
2019,100.00%,6.51%,,,,
2020,100.00%,,,,,


In [11]:
# Проверим расчет retention, если неверно указана детализация
retantion('2020-09-01', '2020-09-24', 'ошибка')

Переменная details задана неверно


In [12]:
# Проверим расчет retention, если даты перепутаны местами
retantion('2020-09-24', '2020-09-01', 'day') 

Даты перепутаны местами


In [13]:
# Проверим расчет retention, если указать неверно первую дату
retantion('1997-01-01', '2020-09-01', 'year') 

Переменная date_start задана неверно. Дата первой регистрации: 1998-11-18


In [14]:
# Проверим расчет retention, если указать неверно вторую дату
retantion('2000-01-01', '2022-09-01', 'year')

Переменная date_end задана неверно. Дата последнего захода пользователя в игру: 2020-09-23
